In [ ]:
import transformers
from transformers import AutoTokenizer, AutoModel

In [ ]:
model_name = "google-bert/bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
text = "hey here's how you can train this model"

In [ ]:
tokens = tokenizer(text)

In [ ]:
print(tokens['input_ids']) #larger number -> rarity of the word.

[101, 23998, 1303, 112, 188, 1293, 1128, 1169, 2669, 1142, 2235, 102]


In [ ]:
print(tokens)

{'input_ids': [101, 23998, 1303, 112, 188, 1293, 1128, 1169, 2669, 1142, 2235, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
tokens2 = tokenizer.encode(text)

In [ ]:
print( tokens2)

[101, 23998, 1303, 112, 188, 1293, 1128, 1169, 2669, 1142, 2235, 102]


In [ ]:
tokens3 = tokenizer.convert_ids_to_tokens(tokens['input_ids'])

print("Tokens:", tokens3)

Tokens: ['[CLS]', 'hey', 'here', "'", 's', 'how', 'you', 'can', 'train', 'this', 'model', '[SEP]']


In [ ]:
model = transformers.BertModel.from_pretrained(model_name)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [ ]:
embedding_layer = model.embeddings

In [ ]:
word_embeddings = embedding_layer.word_embeddings

print(word_embeddings.weight)

Parameter containing:
tensor([[-0.0005, -0.0416,  0.0131,  ..., -0.0039, -0.0335,  0.0150],
        [ 0.0169, -0.0311,  0.0042,  ..., -0.0147, -0.0356, -0.0036],
        [-0.0006, -0.0267,  0.0080,  ..., -0.0100, -0.0331, -0.0165],
        ...,
        [-0.0064,  0.0166, -0.0204,  ..., -0.0418, -0.0492,  0.0042],
        [-0.0048, -0.0027, -0.0290,  ..., -0.0512,  0.0045, -0.0118],
        [ 0.0313, -0.0297, -0.0230,  ..., -0.0145, -0.0525,  0.0284]],
       requires_grad=True)


In [ ]:
word_embeddings.weight.shape

torch.Size([28996, 768])

In [ ]:
word1 = 'smart'
print(tokenizer(word1))

{'input_ids': [101, 6866, 102], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}


In [ ]:
word2 = 'intelligent'
print(tokenizer(word2))

{'input_ids': [101, 9998, 102], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}


In [ ]:
#compute cosine sim of the words
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
#remove CLS and SEP tokes for this computation
token_ids1 = tokenizer.encode(word1, add_special_tokens=False)
token_ids2 = tokenizer.encode(word2, add_special_tokens=False)

In [ ]:
embedding1 = embedding_layer.word_embeddings.weight[token_ids1]
embedding2 = embedding_layer.word_embeddings.weight[token_ids2]

In [ ]:
embedding1 = embedding1.squeeze().detach().numpy()
embedding2 = embedding2.squeeze().detach().numpy()

In [ ]:
magnitude1 = np.linalg.norm(embedding1)
magnitude2 = np.linalg.norm(embedding2)

cosine_sim = np.dot(embedding1, embedding2)/ (magnitude1*magnitude2)

In [ ]:
cosine_sim = cosine_similarity([embedding1], [embedding2])
print(f"Cosine sim of '{word1}' and '{word2}'")
print(cosine_sim)

Cosine sim of 'smart' and 'intelligent'
[[0.46157777]]


In [ ]:
import torch

model = AutoModel.from_pretrained(model_name)

In [ ]:
token_ids1 = tokenizer(word1, return_tensors="pt", add_special_tokens=False)
token_ids2 = tokenizer(word2, return_tensors="pt", add_special_tokens=False)

with torch.no_grad():
    outputs1 = model(**token_ids1)
    outputs2 = model(**token_ids2)

embedding1 = outputs1.last_hidden_state.mean(dim=1).squeeze().numpy()
embedding2 = outputs2.last_hidden_state.mean(dim=1).squeeze().numpy()

cosine_sim = cosine_similarity([embedding1], [embedding2])
print(f"Cosine sim of '{word1}', '{word2}'")
print(cosine_sim)

Cosine sim of 'smart', 'intelligent'
[[0.9999985]]


In [ ]:
#try comparing the similarity of the CLs tok of each word instead.
#CLS token can also be used to rep the entire sentence
token_ids1 = tokenizer(word1, return_tensors="pt", add_special_tokens=True)
token_ids2 = tokenizer(word2, return_tensors="pt")

with torch.no_grad():
    outputs1 = model(**token_ids1)
    outputs2 = model(**token_ids2)

embedding1 = outputs1.last_hidden_state[0,0].numpy()
embedding2 = outputs2.last_hidden_state[0,0].numpy()

cosine_sim = cosine_similarity([embedding1], [embedding2])
print(f"Cosine sim of '{word1}', '{word2}'")
print(cosine_sim)

Cosine sim of 'smart', 'intelligent'
[[0.9586766]]


In [ ]:
#now for sentences..
sent1 = "Subway sells sandwiches."
sent2 = "Mcdonalds sells burgers."

print(tokenizer(sent1))
print(tokenizer(sent2))

{'input_ids': [101, 21925, 16695, 25326, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1]}
{'input_ids': [101, 150, 1665, 3842, 18728, 1116, 16695, 171, 23872, 1116, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
sent_ids1 = tokenizer(sent1, return_tensors="pt", padding=True)
sent_ids2 = tokenizer(sent2, return_tensors="pt", padding=True)

In [ ]:
with torch.no_grad():
    outputs1 = model(**sent_ids1)
    outputs2 = model(**sent_ids2)

embedding1 = outputs1.last_hidden_state[0,0].numpy()
embedding2 = outputs2.last_hidden_state[0,0].numpy()

cosine_sim = cosine_similarity([embedding1], [embedding2])
print(f"Cosine sim of '{sent1}', '{sent2}'")
print(cosine_sim)

Cosine sim of 'Subway sells sandwiches.', 'Mcdonoalds sells burgers.'
[[0.9847667]]
